In [1]:
%cd ..

a:\Phong\Crawl_buffett


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time, os
from src.investing import Investing
import pandas as pd
import numpy as np

In [ ]:
df_all = pd.read_csv('A:\Phong\Crawl_buffett\docs\List_company_23052023 - Listing.csv')

path_save = 'data'
miss_count = 0

def check_miss(id_company, path_save, miss_count):
    if not os.path.exists(f"{path_save}/{id_company}/docs/check.csv"):
        return miss_count
    df = pd.read_csv( f"{path_save}/{id_company}/docs/check.csv")  # read file
    for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
        miss_count += (df[f'Link_{quarter}'] != "[]").sum() - (df[f'download_{quarter}'] == "OK").sum()
    if miss_count > 0:
        print(id_company)
    return miss_count
for id_company in df_all['Symbol']:
    miss_count = check_miss(id_company, path_save, miss_count)
print(miss_count)

In [30]:
def check_miss_file(symbol):
    df = pd.read_csv(fr'Data\{symbol}\docs\check.csv')
    QUARTER = ['Q1', 'Q2', 'Q3', 'Q4']
    df_temp = df[['download_Q1', 'download_Q2', 'download_Q3', 'download_Q4']]
    rows, cols = np.where(df_temp.isna())
    lst_time, lst_quarter = [],  []
    for row, col in zip(rows, cols):
        if df[f'Link_{QUARTER[col]}'][row] != '[]':
            lst_time.append(df['Year'][row])
            lst_quarter.append(QUARTER[col])

    df_new = pd.DataFrame({
                'Symbol': symbol,
                    'Year':lst_time,
                'Quarter': lst_quarter,
                })
    return df_new

df_all = pd.read_csv('A:\Phong\Crawl_buffett\docs\List_company_23052023 - Listing.csv')
df_miss = pd.DataFrame(columns=['Symbol', 'Year', 'Quarter'])
for symbol in df_all['Symbol']:
    if os.path.exists(fr'Data\{symbol}\docs\check.csv'):
        df_temp = check_miss_file(symbol)
        df_miss = pd.concat([df_miss, df_temp])
df_miss['check'] = np.nan
df_miss.to_csv('docs\miss_True.csv', index=False)

In [8]:
reverse = True
def get_link(symbol, year, quarter):
    df_check = pd.read_csv(fr'Data\{symbol}\docs\check.csv')
    links = eval(df_check[f'Link_{quarter}'][df_check['Year'] == year].iloc[0])
    times = eval(df_check[f'Time_{quarter}'][df_check['Year'] == year].iloc[0])
    for i, time in enumerate(times):
        if "(訂正)" not in time:
            return links[i], time
        
def thread_file():
    df_miss = pd.read_csv(f'docs\miss_{reverse}.csv')
    id = df_miss[df_miss['check'] != 'OK'].index[0]
    symbol = df_miss['Symbol'][id]
    year = df_miss['Year'][id]
    quarter = df_miss['Quarter'][id]
    df_miss['check'][id] = 'Doing'
    df_miss.to_csv(f'docs\miss_{reverse}.csv', index=False)
    link, time = get_link(symbol, year, quarter)
    return link, time

thread_file()

('https://www.buffett-code.com/company/3928/library/3e54c1d3a782a5d880224c/preview',
 '決算短信 (2022/5/13)')